<a href="https://colab.research.google.com/github/hijyun/dalc-mini/blob/master/1wk/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TA-Lib 설치 코드
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3
import talib

# 야후 파이낸스설치
!pip install yfinance
import yfinance as yf

import pandas as pd
import numpy as np

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3689    0  3689    0     0   3783      0 --:--:-- --:--:-- --:--:--  3779
100  503k  100  503k    0     0   183k      0  0:00:02  0:00:02 --:--:--  368k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3697    0  3697    0     0   4401      0 --:--:-- --:--:-- --:--:--  4395
100  406k  100  406k    0     0   148k      0  0:00:02  0:00:02 --:--:--  372k
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 8.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lx

In [ ]:
# 네이버 데이터 불러오기
naver_df= df = yf.download('090430.KS',start = '2005-01-01') # 원하는 주식의 종목코드+.KS
sox_df = yf.download('^SOX', start = '2000-01-01')
vix_df = yf.download('^VIX', start = '2000-01-01')
snp500_df = yf.download('^GSPC', start = '2000-01-01')
kospi_df= yf.download('^KS11', start = '2000-01-01')

kospi_df.to_csv('kospi.csv')
naver_df.to_csv('naver.csv')
sox_df.to_csv('sox_df.csv')
vix_df.to_csv('vix_df.csv')
snp500_df.to_csv('s&p500.csv')

naver_df = pd.read_csv('naver.csv',index_col='Date', parse_dates=True)
sox_df = pd.read_csv('sox_df.csv', index_col = 'Date', parse_dates = True)
vix_df = pd.read_csv('vix_df.csv', index_col = 'Date', parse_dates = True)
snp500_df = pd.read_csv('s&p500.csv', index_col = 'Date', parse_dates = True)
kospi_df = pd.read_csv('kospi.csv', index_col = 'Date', parse_dates = True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
snp500_df = snp500_df.loc[:,['Close']].copy()
snp500_df.rename(columns={'Close':'S&P500'},inplace=True)
sox_df = sox_df.loc[:,['Close']].copy()
sox_df.rename(columns={'Close':'SOX'},inplace=True)
vix_df = vix_df.loc[:,['Close']].copy()
vix_df.rename(columns={'Close':'VIX'},inplace=True)


df = df.join(snp500_df,how='left')
df = df.join(sox_df,how='left')
df = df.join(vix_df,how='left')

In [ ]:
kospi_df['KOSPI'] = kospi_df['Close'].copy()

In [ ]:

kospi_df['kosATR'] = talib.ATR(kospi_df['High'], kospi_df['Low'], kospi_df['Close'], timeperiod=14)

kospi_df['kosADX14'] = talib.ADXR(kospi_df['High'], kospi_df['Low'], kospi_df['Close'], timeperiod=14)


kospi_df = kospi_df.drop(['Open','High','Low','Close','Adj Close','Volume'],axis=1)
df = df.join(kospi_df, how = 'left')

In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume,S&P500,SOX,VIX,KOSPI,kosATR,kosADX14
Date,,,,,,,,,,,,
2006-06-29,38000.0,40500.0,37500.0,39200.0,36039.304688,1195650,1272.869995,446.170013,13.030000,1263.020020,26.263829,42.600569
2006-06-30,39600.0,40950.0,39000.0,40900.0,37602.234375,472510,1270.199951,441.720001,13.080000,1295.150024,26.702844,42.156915
2006-07-03,41300.0,41700.0,39900.0,41400.0,38061.925781,207420,1280.189941,446.010010,13.050000,1294.609985,25.753358,41.413014
2006-07-04,41550.0,44300.0,41550.0,44000.0,40452.289062,277000,NaN,NaN,NaN,1285.920044,25.371692,40.547532
2006-07-05,42700.0,44000.0,42700.0,44000.0,40452.289062,137000,1270.910034,431.899994,14.150000,1279.849976,25.477283,39.137870
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-07,163000.0,164500.0,159000.0,160500.0,160500.000000,269295,4677.029785,3796.320068,18.760000,2954.889893,35.404868,12.853573
2022-01-10,159500.0,159500.0,144000.0,152000.0,152000.000000,897775,4670.290039,3805.389893,19.400000,2926.719971,36.018091,13.029584
2022-01-11,150500.0,153500.0,150000.0,153000.0,153000.000000,249170,4713.069824,3875.439941,18.410000,2927.379883,35.856084,13.288158


In [ ]:
# 기술지표 가공
df['log_return'] = np.log(1 + df['Adj Close'].pct_change())
df['CCI'] = talib.CCI(df['High'], df['Low'], df['Adj Close'], timeperiod=14)


#1.RA : Standard deviation rolling average
# Moving Average
df['MA10'] = talib.SMA(df['Close'],timeperiod=10)
df['MA60'] = talib.SMA(df['Close'],timeperiod=60)

df['MA60_volumne'] = talib.SMA(df['Volume'], timeperiod = 60)
df['RASD5'] = talib.SMA(talib.STDDEV(df['Close'], timeperiod=5, nbdev=1),timeperiod=5)
df['RASD10'] = talib.SMA(talib.STDDEV(df['Close'], timeperiod=5, nbdev=1),timeperiod=10)

#2.MACD : Moving Average Convergence/Divergence
macd, macdsignal, macdhist = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['MACD'] = macd 

# Momentum Indicators
#3.CCI : Commodity Channel Index

df['CCI60'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=60)
df['CCI60'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=90)
# Volatility Indicators 

#4.ATR : Average True Range
df['ATR14'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)

#7.MTM
df['MTM3'] = talib.MOM(df['Close'], timeperiod=3)

#8.ROC : Rate of change : ((price/prevPrice)-1)*100
df['ROC60'] = talib.ROC(df['Close'], timeperiod=60)
df['ROC90'] = talib.ROC(df['Close'], timeperiod=30)

#9.WPR : william percent range (Williams' %R)
df['WPR14'] = talib.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14)

#10.ADXR
df['ADXR14'] = talib.ADXR(df['High'], df['Low'], df['Close'], timeperiod=14)

#11.ADX
df['ADX14'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

In [ ]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,S&P500,SOX,VIX,KOSPI,kosATR,kosADX14,log_return,CCI,MA5,MA10,MA60,MA60_volumne,RASD5,RASD10,MACD,CCI60,ATR14,MTM3,ROC60,ROC90,WPR14,ADXR14,ADX14
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006-06-29,38000.0,40500.0,37500.0,39200.0,36039.304688,1195650,1272.869995,446.170013,13.03,1263.020020,26.263829,42.600569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-06-30,39600.0,40950.0,39000.0,40900.0,37602.234375,472510,1270.199951,441.720001,13.08,1295.150024,26.702844,42.156915,0.042453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-07-03,41300.0,41700.0,39900.0,41400.0,38061.925781,207420,1280.189941,446.010010,13.05,1294.609985,25.753358,41.413014,0.012151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-07-04,41550.0,44300.0,41550.0,44000.0,40452.289062,277000,NaN,NaN,NaN,1285.920044,25.371692,40.547532,0.060909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.0,NaN,NaN,NaN,NaN,NaN
2006-07-05,42700.0,44000.0,42700.0,44000.0,40452.289062,137000,1270.910034,431.899994,14.15,1279.849976,25.477283,39.137870,0.000000,NaN,41900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3100.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv('newquant_amore.csv')

In [ ]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,S&P500,SOX,VIX,KOSPI,kosATR,kosADX14,log_return,CCI,MA5,MA10,MA60,MA60_volumne,RASD5,RASD10,MACD,CCI60,ATR14,MTM3,ROC60,ROC90,WPR14,ADXR14,ADX14
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006-06-29,38000.0,40500.0,37500.0,39200.0,36039.304688,1195650,1272.869995,446.170013,13.03,1263.020020,26.263829,42.600569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-06-30,39600.0,40950.0,39000.0,40900.0,37602.234375,472510,1270.199951,441.720001,13.08,1295.150024,26.702844,42.156915,0.042453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-07-03,41300.0,41700.0,39900.0,41400.0,38061.925781,207420,1280.189941,446.010010,13.05,1294.609985,25.753358,41.413014,0.012151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-07-04,41550.0,44300.0,41550.0,44000.0,40452.289062,277000,NaN,NaN,NaN,1285.920044,25.371692,40.547532,0.060909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.0,NaN,NaN,NaN,NaN,NaN
2006-07-05,42700.0,44000.0,42700.0,44000.0,40452.289062,137000,1270.910034,431.899994,14.15,1279.849976,25.477283,39.137870,0.000000,NaN,41900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3100.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3850 entries, 2006-06-29 to 2022-01-13
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          3850 non-null   float64
 1   High          3850 non-null   float64
 2   Low           3850 non-null   float64
 3   Close         3850 non-null   float64
 4   Adj Close     3850 non-null   float64
 5   Volume        3850 non-null   int64  
 6   S&P500        3734 non-null   float64
 7   SOX           3734 non-null   float64
 8   VIX           3734 non-null   float64
 9   KOSPI         3838 non-null   float64
 10  kosATR        3838 non-null   float64
 11  kosADX14      3838 non-null   float64
 12  log_return    3849 non-null   float64
 13  CCI           3837 non-null   float64
 14  MA5           3846 non-null   float64
 15  MA10          3841 non-null   float64
 16  MA60          3791 non-null   float64
 17  MA60_volumne  3791 non-null   float64
 18  RASD5     